In [31]:
# Dependencies and Setup
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
from scipy.stats import zscore
from scipy.stats import stats
import os
import seaborn as sns
import re

from statsmodels.compat import lzip
import statsmodels.api as sm
from statsmodels.formula.api import ols

In [58]:
# Files to Load (csv files downloaded from the internet)

file1 = "data/population-density-vs-prosperity.csv"
file2 = "data/urban-and-rural-population.csv"
file3 = "data/urban-vs-rural-majority.csv"
file4 = "data/life-expectancy-at-birth-total-years.csv"
file5 = "data/median-age.csv"
file6 = "data/six-dem_mod.csv"
file7 = "data/covid-confirmed-cases-since-100th-case_hkg.csv"

# Read File and store into Pandas data frame

density_vs_prosperity = pd.read_csv(file1)
urban_rural_population = pd.read_csv(file2)
urban_vs_rural_majority = pd.read_csv(file3)
life_expectancy = pd.read_csv(file4)
median_age = pd.read_csv(file5)
hoefstede = pd.read_csv(file6)
covid_data = pd.read_csv(file7)

In [33]:
density_vs_prosperity.shape

(47234, 6)

In [34]:
density_vs_prosperity.dropna(subset=['Code'],inplace=True)
density_vs_prosperity.shape

(44392, 6)

In [35]:
urban_rural_population.dropna(subset=['Code'],inplace=True)
urban_rural_population.shape

(12363, 5)

In [36]:
urban_vs_rural_majority.dropna(subset=['Code'],inplace=True)
urban_vs_rural_majority.shape

(24019, 5)

In [37]:
life_expectancy.dropna(subset=['Code'],inplace=True)
life_expectancy.shape

(11125, 4)

In [38]:
median_age.dropna(subset=['Code'],inplace=True)
median_age.shape

(6324, 4)

In [59]:
covid_data.shape

(12325, 5)

In [101]:
# Merge data
merged_data = pd.merge(density_vs_prosperity, urban_rural_population, on=["Entity","Code", "Year"])
merged_data.shape

(14905, 8)

In [102]:
merged_data = pd.merge(merged_data, urban_vs_rural_majority, on=["Entity", "Code", "Year"])
merged_data.shape

(12340, 10)

In [103]:
merged_data = pd.merge(merged_data, life_expectancy, on=["Entity", "Code", "Year"])
merged_data.shape

(11063, 11)

In [104]:
merged_data.head()

,Entity,Code,Year,Population density (people per km² of land area),GDP per capita (constant 2011 international $),Total population (Gapminder),Urban population,Rural population,Urban (%),Rural (%),"Life expectancy at birth, total (years) (years)"
0,Afghanistan,AFG,1960,NaN,NaN,8996351.0,755783,8240568,8.401,91.598999,32.292
1,Afghanistan,AFG,1961,14.040934,NaN,9166764.0,796042,8370722,8.684,91.316002,32.742
2,Afghanistan,AFG,1962,14.315271,NaN,9345868.0,838885,8506983,8.976,91.024002,33.185
3,Afghanistan,AFG,1963,14.603367,NaN,9533954.0,884370,8649584,9.276,90.723999,33.624
4,Afghanistan,AFG,1964,14.905739,NaN,9731361.0,932848,8798513,9.586,90.414001,34.060


In [105]:
merged_data = pd.merge(merged_data, median_age, on=["Entity", "Code", "Year"])
merged_data.shape

(2291, 12)

In [106]:
#Format columns
merged_data = merged_data.rename(columns={"Population density (people per km² of land area)": "pop_density",
                                          "GDP per capita (constant 2011 international $)" : "gdp",
                                         "Total population (Gapminder)": "total_pop",
                                         "Urban population": "urban_pop",
                                         "Rural population": "rural_pop",
                                         "Urban (%)":"urban_pct",
                                         "Rural (%)": "rural_pct",
                                         "Life expectancy at birth, total (years) (years)":"life_expectancy",
                                         "UN Population Division (Median Age) (2017) (years)":"median_age"})
merged_data.head()

,Entity,Code,Year,pop_density,gdp,total_pop,urban_pop,rural_pop,urban_pct,rural_pct,life_expectancy,median_age
0,Afghanistan,AFG,1960,NaN,NaN,8996351.0,755783,8240568,8.401,91.598999,32.292,18.799999
1,Afghanistan,AFG,1965,15.222887,NaN,9938414.0,984301,8954113,9.904,90.096001,34.495,18.400000
2,Afghanistan,AFG,1970,17.042127,NaN,11126123.0,1295415,9830708,11.643,88.357002,36.678,17.900000
3,Afghanistan,AFG,1975,19.284818,NaN,12590286.0,1717441,10872845,13.641,86.359001,39.039,17.400000
4,Afghanistan,AFG,1980,20.292819,NaN,13248370.0,2119077,11129293,15.995,84.004997,41.853,17.000000


In [107]:
hoefstede.shape

(127, 8)

In [108]:
hoefstede.head()

,Code,Entity,pdi,idv,mas,uai,ltowvs,ivr
0,GTM,Guatemala,95,6,37,101,#NULL!,#NULL!
1,ECU,Ecuador,78,8,63,67,#NULL!,#NULL!
2,PAN,Panama,95,11,44,86,#NULL!,#NULL!
3,VEN,Venezuela,81,12,73,76,16,100
4,COL,Colombia,67,13,64,80,13,83


In [109]:
hoefstede = hoefstede.rename(columns={"ctr": "Code",
                                          "country" : "Entity"})
hoefstede.head()

,Code,Entity,pdi,idv,mas,uai,ltowvs,ivr
0,GTM,Guatemala,95,6,37,101,#NULL!,#NULL!
1,ECU,Ecuador,78,8,63,67,#NULL!,#NULL!
2,PAN,Panama,95,11,44,86,#NULL!,#NULL!
3,VEN,Venezuela,81,12,73,76,16,100
4,COL,Colombia,67,13,64,80,13,83


In [110]:
covid_data.shape

(12325, 5)

In [111]:
merged_data = pd.merge(merged_data, hoefstede, on=[ "Code"])
merged_data.head()

,Entity_x,Code,Year,pop_density,gdp,total_pop,urban_pop,rural_pop,urban_pct,rural_pct,life_expectancy,median_age,Entity_y,pdi,idv,mas,uai,ltowvs,ivr
0,Albania,ALB,1960,NaN,NaN,1636056.0,493982,1114818,30.705,69.294998,62.279,20.200001,Albania,90,20,80,70,61,15
1,Albania,ALB,1965,68.058066,NaN,1896125.0,582374,1282417,31.230,68.769997,65.848,19.700001,Albania,90,20,80,70,61,15
2,Albania,ALB,1970,77.937190,NaN,2150598.0,677801,1457678,31.740,68.260002,66.933,19.600000,Albania,90,20,80,70,61,15
3,Albania,ALB,1975,87.767555,NaN,2411229.0,786668,1618163,32.712,67.288002,68.734,20.200001,Albania,90,20,80,70,61,15
4,Albania,ALB,1980,97.518139,NaN,2681239.0,902120,1769877,33.762,66.237999,70.207,21.400000,Albania,90,20,80,70,61,15


In [112]:
merged_data.shape

(1381, 19)

In [113]:
merged_data['Year'].max()

2015

In [114]:
merged_data = merged_data[merged_data["Year"] == 2015]
merged_data.head()

,Entity_x,Code,Year,pop_density,gdp,total_pop,urban_pop,rural_pop,urban_pct,rural_pct,life_expectancy,median_age,Entity_y,pdi,idv,mas,uai,ltowvs,ivr
11,Albania,ALB,2015,105.135146,10970.452245,2923352.0,1654503,1226200,57.434,42.566002,78.174,36.200001,Albania,90,20,80,70,61,15
23,Algeria,DZA,2015,16.740504,13724.723853,39871528.0,28248180,11623348,70.848,29.152000,75.860,27.500000,Algeria,#NULL!,#NULL!,#NULL!,#NULL!,26,32
35,Angola,AGO,2015,22.346439,6227.664689,27859305.0,17675615,10183690,63.446,36.554001,61.241,16.400000,Angola,83,18,20,60,15,83
47,Argentina,ARG,2015,15.865065,19116.907915,43417765.0,39728558,3689207,91.503,8.497000,76.415,30.799999,Argentina,49,46,56,86,20,62
59,Armenia,ARM,2015,102.456972,8195.933650,2916950.0,1840158,1076792,63.085,36.915001,74.445,33.900002,Armenia,#NULL!,#NULL!,#NULL!,#NULL!,61,#NULL!


In [115]:
merged_data.drop(['Entity_y','Year'], axis=1,inplace=True)

In [116]:
merged_data = merged_data.rename(columns={"Entity_x": "Entity"})

In [117]:
merged_data.head()

,Entity,Code,pop_density,gdp,total_pop,urban_pop,rural_pop,urban_pct,rural_pct,life_expectancy,median_age,pdi,idv,mas,uai,ltowvs,ivr
11,Albania,ALB,105.135146,10970.452245,2923352.0,1654503,1226200,57.434,42.566002,78.174,36.200001,90,20,80,70,61,15
23,Algeria,DZA,16.740504,13724.723853,39871528.0,28248180,11623348,70.848,29.152000,75.860,27.500000,#NULL!,#NULL!,#NULL!,#NULL!,26,32
35,Angola,AGO,22.346439,6227.664689,27859305.0,17675615,10183690,63.446,36.554001,61.241,16.400000,83,18,20,60,15,83
47,Argentina,ARG,15.865065,19116.907915,43417765.0,39728558,3689207,91.503,8.497000,76.415,30.799999,49,46,56,86,20,62
59,Armenia,ARM,102.456972,8195.933650,2916950.0,1840158,1076792,63.085,36.915001,74.445,33.900002,#NULL!,#NULL!,#NULL!,#NULL!,61,#NULL!


In [118]:
merged_data.shape

(116, 17)

In [119]:
covid_data.head()

,Entity,Code,Date,(cases),Number of days since the 100th confirmed case (days)
0,Afghanistan,AFG,31-Dec-19,0.0,NaN
1,Afghanistan,AFG,1-Jan-20,0.0,NaN
2,Afghanistan,AFG,2-Jan-20,0.0,NaN
3,Afghanistan,AFG,3-Jan-20,0.0,NaN
4,Afghanistan,AFG,4-Jan-20,0.0,NaN


In [120]:
merged_data = pd.merge(merged_data, covid_data, on=[ "Code"])
merged_data.head()

,Entity_x,Code,pop_density,gdp,total_pop,urban_pop,rural_pop,urban_pct,rural_pct,life_expectancy,...,pdi,idv,mas,uai,ltowvs,ivr,Entity_y,Date,(cases),Number of days since the 100th confirmed case (days)
0,Albania,ALB,105.135146,10970.452245,2923352.0,1654503,1226200,57.434,42.566002,78.174,...,90,20,80,70,61,15,Albania,9-Mar-20,2.0,NaN
1,Albania,ALB,105.135146,10970.452245,2923352.0,1654503,1226200,57.434,42.566002,78.174,...,90,20,80,70,61,15,Albania,10-Mar-20,6.0,NaN
2,Albania,ALB,105.135146,10970.452245,2923352.0,1654503,1226200,57.434,42.566002,78.174,...,90,20,80,70,61,15,Albania,11-Mar-20,10.0,NaN
3,Albania,ALB,105.135146,10970.452245,2923352.0,1654503,1226200,57.434,42.566002,78.174,...,90,20,80,70,61,15,Albania,12-Mar-20,11.0,NaN
4,Albania,ALB,105.135146,10970.452245,2923352.0,1654503,1226200,57.434,42.566002,78.174,...,90,20,80,70,61,15,Albania,13-Mar-20,23.0,NaN


In [121]:
merged_data.drop(['Entity_y'], axis=1,inplace=True)

In [122]:
merged_data = merged_data.rename(columns={"Entity_x": "Entity",
                                         " (cases)" : "cases",
                                          "Number of days since the 100th confirmed case (days)":"days_gt_100"})

In [123]:
merged_data.shape

(7750, 20)

In [124]:
merged_data.tail()

,Entity,Code,pop_density,gdp,total_pop,urban_pop,rural_pop,urban_pct,rural_pct,life_expectancy,median_age,pdi,idv,mas,uai,ltowvs,ivr,Date,cases,days_gt_100
7745,Zimbabwe,ZWE,40.784415,1912.280261,15777451.0,5109528,10667923,32.385,67.614998,60.398,19.0,#NULL!,#NULL!,#NULL!,#NULL!,15,28,9-Apr-20,11.0,NaN
7746,Zimbabwe,ZWE,40.784415,1912.280261,15777451.0,5109528,10667923,32.385,67.614998,60.398,19.0,#NULL!,#NULL!,#NULL!,#NULL!,15,28,10-Apr-20,11.0,NaN
7747,Zimbabwe,ZWE,40.784415,1912.280261,15777451.0,5109528,10667923,32.385,67.614998,60.398,19.0,#NULL!,#NULL!,#NULL!,#NULL!,15,28,11-Apr-20,11.0,NaN
7748,Zimbabwe,ZWE,40.784415,1912.280261,15777451.0,5109528,10667923,32.385,67.614998,60.398,19.0,#NULL!,#NULL!,#NULL!,#NULL!,15,28,12-Apr-20,14.0,NaN
7749,Zimbabwe,ZWE,40.784415,1912.280261,15777451.0,5109528,10667923,32.385,67.614998,60.398,19.0,#NULL!,#NULL!,#NULL!,#NULL!,15,28,13-Apr-20,14.0,NaN


In [125]:
# Export CSV
merged_data.to_csv('merged_data_modified.csv')  

In [ ]:
# Run a boxplot of the Individualism vs.Rural AAMR

# Set outliers style
flierprops = dict(markerfacecolor='Red', markeredgecolor='black',markersize=8,
              linestyle='none')

# Plot the boxplot
g = sns.boxplot(y='Age-Adjusted Mortality', x='URCS', 
                 data=merged_data, 
                 palette="colorblind",flierprops=flierprops)
g.set_xticklabels(g.get_xticklabels(), rotation=30)